# FPUS23 — YOLO Noise Experiments (n/s/m) — v2

- Trains YOLO11 n/s/m with stronger noise/denoise toggles.
- Saves checkpoints every epoch to Drive and auto-resumes.
- This notebook approximates "noise" via augment knobs; if you
  have a denoiser in `/content/fpus23/scripts/train_denoising_autoencoder.py`,
  run it first and set `USE_DENOISED=True` to switch your image root.


In [ ]:
%pip -q install -U ultralytics >/dev/null
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
from pathlib import Path
USE_DENOISED = False  # set True if you pre-computed denoised images
YOLO_BASE = Path('/content/fpus23_project/dataset/fpus23_yolo')
YOLO_YAML = YOLO_BASE / 'data.yaml'
if USE_DENOISED and (YOLO_BASE / 'images_denoised').exists():
    # If your prepare script writes an alternate denoised layout,
    # swap data.yaml or overwrite its split roots here as needed.
    print('Denoised root detected; using original labels with denoised images')
assert YOLO_YAML.exists(), 'Missing dataset YAML. Prepare dataset first.'
print('Using', YOLO_YAML)


In [ ]:
# Stronger noise-ish augs; save every epoch; auto-resume.
sizes = ['n','s','m']
batches = {'n':16,'s':16,'m':8}
for sz in sizes:
  run = f'fpus23_noise_{sz}'
  !yolo train data=$YOLO_YAML model=yolo11{sz}.pt epochs=100 batch={batches[sz]} imgsz=768 \
       project='/content/drive/MyDrive/FPUS23_runs' name=$run deterministic=True workers=2 \
       auto_augment=randaugment erasing=0.6 degrees=10 translate=0.1 scale=0.5 \
       cls=3.0 cos_lr=True resume=True save_period=1


In [ ]:
import pandas as pd
from pathlib import Path
BASE = Path('/content/drive/MyDrive/FPUS23_runs')
runs = ['fpus23_noise_n','fpus23_noise_s','fpus23_noise_m']
rows = []
for r in runs:
  p = BASE / r / 'results.csv'
  if p.exists():
    df = pd.read_csv(p).dropna(how='all')
    if len(df):
      last = df.tail(1).squeeze()
      rows.append({'run': r, 'epoch': last.get('epoch'),
                   'mAP50': last.get('metrics/mAP50(B)'),
                   'mAP50-95': last.get('metrics/mAP50-95(B)'),
                   'precision': last.get('metrics/precision(B)'),
                   'recall': last.get('metrics/recall(B)'),
                   'csv': str(p)})
if rows:
  import pandas as pd
  display(pd.DataFrame(rows).sort_values('mAP50-95', ascending=False))
else:
  print('No results found yet. Check your Drive runs folder.')
